In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, TimeDistributed, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop, Adam

train_data = pd.read_csv('../data/balanced_train.csv')
test_data = pd.read_csv('../data/balanced_test.csv')

genre_columns = train_data.columns.drop(['Name', 'Description Tokenized'])

# Tokenizar os textos
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(train_data['Description Tokenized']) + list(test_data['Description Tokenized']))
word_index = tokenizer.word_index
num_words = len(word_index) + 1

X_train = tokenizer.texts_to_sequences(train_data['Description Tokenized'])
X_test = tokenizer.texts_to_sequences(test_data['Description Tokenized'])

maxlen = max(max([len(sequence) for sequence in X_train]), max([len(sequence) for sequence in X_test]))

# Padronizar os textos
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

#labels one-hot encoded
y_train = train_data[genre_columns].values
y_test = test_data[genre_columns].values


# modelo LSTM unidirecional
embedding_dim = 100

model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=maxlen))
model.add(LSTM(128, return_sequences=True, dropout=0.25, recurrent_dropout=0.25))
model.add(TimeDistributed(Dense(128, activation='relu')))
model.add(Dropout(0.5))
model.add(LSTM(128, dropout=0.25, recurrent_dropout=0.25))
model.add(Dense(len(genre_columns), activation='sigmoid'))

# Compilar e treinar o modelo
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

num_epochs = 15
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), verbose=2)

# 4. Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

Epoch 1/15
160/160 - 69s - loss: 0.5638 - accuracy: 0.2670 - val_loss: 0.5586 - val_accuracy: 0.2648 - 69s/epoch - 432ms/step
Epoch 2/15
160/160 - 57s - loss: 0.5580 - accuracy: 0.2649 - val_loss: 0.5577 - val_accuracy: 0.2648 - 57s/epoch - 359ms/step
Epoch 3/15
160/160 - 54s - loss: 0.5406 - accuracy: 0.2745 - val_loss: 0.5178 - val_accuracy: 0.3141 - 54s/epoch - 340ms/step
Epoch 4/15
160/160 - 46s - loss: 0.4960 - accuracy: 0.3147 - val_loss: 0.4963 - val_accuracy: 0.3195 - 46s/epoch - 287ms/step
Epoch 5/15
160/160 - 42s - loss: 0.4568 - accuracy: 0.3405 - val_loss: 0.4679 - val_accuracy: 0.3242 - 42s/epoch - 261ms/step
Epoch 6/15
160/160 - 48s - loss: 0.4124 - accuracy: 0.3659 - val_loss: 0.4455 - val_accuracy: 0.3781 - 48s/epoch - 297ms/step
Epoch 7/15
160/160 - 55s - loss: 0.3722 - accuracy: 0.3843 - val_loss: 0.4453 - val_accuracy: 0.3625 - 55s/epoch - 341ms/step
Epoch 8/15
160/160 - 41s - loss: 0.3313 - accuracy: 0.3868 - val_loss: 0.4518 - val_accuracy: 0.3273 - 41s/epoch - 257

In [2]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_binarized = np.round(y_pred)

report = classification_report(y_test, y_pred_binarized, target_names=genre_columns, zero_division=0)
print(report)

40/40 [==============================] - 4s 39ms/step
                         precision    recall  f1-score   support

                 Comedy       0.74      0.66      0.70       487
                  Crime       0.82      0.73      0.77       271
                  Drama       0.68      0.91      0.78       613
                Romance       0.75      0.62      0.68       270
   Action and Adventure       0.78      0.68      0.73       463
Documentary and History       0.58      0.69      0.63       172
   Family and Animation       0.70      0.65      0.68       275
     Fantasy and Sci-Fi       0.70      0.66      0.68       259
    Horror and Thriller       0.71      0.61      0.66       309

              micro avg       0.72      0.71      0.71      3119
              macro avg       0.72      0.69      0.70      3119
           weighted avg       0.72      0.71      0.71      3119
            samples avg       0.71      0.69      0.68      3119



In [4]:
from sklearn.metrics import roc_auc_score, average_precision_score

y_pred_prob = model.predict(X_test)

# Cálculo do AUC-PR e AUC-ROC geral
auc_pr_micro = average_precision_score(y_test, y_pred_prob, average='micro')
auc_pr_macro = average_precision_score(y_test, y_pred_prob, average='macro')
auc_pr_weighted = average_precision_score(y_test, y_pred_prob, average='weighted')

auc_roc_micro = roc_auc_score(y_test, y_pred_prob, average='micro')
auc_roc_macro = roc_auc_score(y_test, y_pred_prob, average='macro')
auc_roc_weighted = roc_auc_score(y_test, y_pred_prob, average='weighted')

# Criação da tabela
data = {'AUC-PR': [auc_pr_micro, auc_pr_macro, auc_pr_weighted],
        'AUC-ROC': [auc_roc_micro, auc_roc_macro, auc_roc_weighted]}
index = ['micro', 'macro', 'weighted']
results = pd.DataFrame(data, index=index)

# Exibição dos resultados
print(results)

40/40 [==============================] - 2s 39ms/step
            AUC-PR   AUC-ROC
micro     0.753540  0.884976
macro     0.752009  0.872386
weighted  0.756798  0.863066


In [5]:
from sklearn.metrics import precision_recall_curve, roc_curve, auc

pr_curves = {}
roc_curves = {}
for i, genre in enumerate(genre_columns):
    precision, recall, _ = precision_recall_curve(y_test[:, i], y_pred_prob[:, i])
    fpr, tpr, _ = roc_curve(y_test[:, i], y_pred_prob[:, i])
    pr_curves[genre] = (precision, recall)
    roc_curves[genre] = (fpr, tpr)
    
auc_pr = []
auc_roc = []
for i, genre in enumerate(genre_columns):
    ap = auc(pr_curves[genre][1], pr_curves[genre][0])
    ar = auc(roc_curves[genre][0], roc_curves[genre][1])
    auc_pr.append(ap)
    auc_roc.append(ar)

results = pd.DataFrame({'genre': genre_columns, 'AUC-PR': auc_pr, 'AUC-ROC': auc_roc})
print(results)


                     genre    AUC-PR   AUC-ROC
0                   Comedy  0.780158  0.844090
1                    Crime  0.816230  0.911106
2                    Drama  0.749322  0.815749
3                  Romance  0.740574  0.890854
4     Action and Adventure  0.796595  0.866368
5  Documentary and History  0.722892  0.891482
6     Family and Animation  0.748108  0.880988
7       Fantasy and Sci-Fi  0.721744  0.888144
8      Horror and Thriller  0.685137  0.862691
